In [11]:
import cv2
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from random import shuffle
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
import numpy as np

In [12]:
#path of the folder containing all the foreground binary images
train_data='D:\\xy'

In [13]:
#storing the image and label of train data

def train_data_with_label():
    train_images=[]
    global ohl
    for i in tqdm(os.listdir(train_data)):
        path=os.path.join(train_data,i)
        img=cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img=cv2.resize(img,(64,64))
        label=path.split('+')[1]
        if label=='wave1.jpg.jpg':
            ohl=np.array([1,0,0])
        if label=='wave2.jpg.jpg':
            ohl=np.array([0,1,0])
        if label=='bend.jpg.jpg':
            ohl=np.array([0,0,1])
        train_images.append([np.array(img),ohl])
    shuffle(train_images)
    return train_images

training_images = train_data_with_label()

tr_img_data = np.array([i[0] for i in training_images]).reshape(-1,64,64,1)
tr_lbl_data = np.array([i[1] for i in training_images])




100%|███████████████████████████████████████████████████████████████████████████████| 551/551 [00:02<00:00, 253.31it/s]


In [14]:
#cnn model

model=Sequential()
model.add(InputLayer(input_shape=[64,64,1]))
model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))         
model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))         
model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
model.add(MaxPool2D(pool_size=5,padding='same'))          
model.add(Dropout(0.25)) #avoid overfitting
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(3,activation='softmax')) #3 labels
optimizer=Adam(lr=1e-3)


model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(x=tr_img_data,y=tr_lbl_data,epochs=10,batch_size=100)
model.summary()

Epoch 1/10
551/551 [==============================] - ETA: 14s - loss: 10.3059 - acc: 0.310 - ETA: 11s - loss: 8.8173 - acc: 0.360 - ETA: 7s - loss: 8.7880 - acc: 0.3467 - ETA: 4s - loss: 8.7762 - acc: 0.352 - ETA: 1s - loss: 8.6233 - acc: 0.358 - 18s 33ms/step - loss: 8.3585 - acc: 0.3684
Epoch 2/10
551/551 [==============================] - ETA: 14s - loss: 6.6342 - acc: 0.40 - ETA: 11s - loss: 6.6061 - acc: 0.41 - ETA: 8s - loss: 6.4469 - acc: 0.4367 - ETA: 4s - loss: 6.1023 - acc: 0.462 - ETA: 1s - loss: 6.0116 - acc: 0.460 - 18s 32ms/step - loss: 5.8772 - acc: 0.4701
Epoch 3/10
551/551 [==============================] - ETA: 14s - loss: 4.3041 - acc: 0.52 - ETA: 10s - loss: 3.7412 - acc: 0.55 - ETA: 7s - loss: 3.8774 - acc: 0.5600 - ETA: 4s - loss: 3.5015 - acc: 0.572 - ETA: 1s - loss: 3.3361 - acc: 0.590 - 17s 31ms/step - loss: 3.2520 - acc: 0.5953
Epoch 4/10
551/551 [==============================] - ETA: 13s - loss: 1.6098 - acc: 0.69 - ETA: 10s - loss: 1.4442 - acc: 0.69 - ETA

In [31]:
#path of the video to be tested
path="D:\\wave1\\daria_wave1.avi"
cam = cv2.VideoCapture(path) 

#path of the background of the above video
img=cv2.imread("D:\\hj.jpg",cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(64,64))
  
while(True): 
              
        ret,frame = cam.read() 
 
        if ret==True:     
            
            cv2.imwrite("D:\\frame.jpg",frame)
            image2 = cv2.imread("D:\\frame.jpg",cv2.IMREAD_GRAYSCALE)
            image2=cv2.resize(image2,(64,64))
            imgg=cv2.subtract(img,image2)
            cv2.imwrite("D:\\hjg.jpg",imgg)
        
            blur = cv2.GaussianBlur(imgg,(3,3),0)
            ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            cv2.imwrite("D:\\hji.jpg",th3)
            data=th3.reshape(1,64,64,1)
            
            #feeding the processed image to model for prediction
            model_out=model.predict([data]) 
            
            if np.argmax(model_out)==0:
                label="1HandWave"
            if np.argmax(model_out)==1:
                label="2HandWave"
            if np.argmax(model_out)==2:
                label="bending"
                
            #displaying each frame
            cv2.putText(frame,label,(0,25),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255))
            cv2.imshow("frame",frame)
 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                 break
        else:
            break
       
  
# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows() 




In [33]:
#testing a live video
#for first few seconds the backround should be left empty
cam = cv2.VideoCapture(0) 
c=0
  
while(cam.isOpened()): 
      
    # reading from frame 
    
        ret,frame = cam.read() 
 
        if ret==True: 
            
            #saving the image of background only
            if c==0:
                cv2.imwrite("D:\\hj.jpg",frame)
                img=cv2.imread("D:\\hj.jpg",cv2.IMREAD_GRAYSCALE)
            img=cv2.resize(img,(64,64))
            cv2.imwrite("D:\\frame.jpg",frame)
            image2 = cv2.imread("D:\\frame.jpg",cv2.IMREAD_GRAYSCALE)
            image2=cv2.resize(image2,(64,64))
            
            #subtracting the background from the frame
            imgg=cv2.subtract(img,image2)
        
            blur = cv2.GaussianBlur(imgg,(3,3),0)
            ret3,th3 = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            cv2.imwrite("D:\\hji.jpg",th3)
            data=th3.reshape(1,64,64,1)
            model_out=model.predict([data]) 
            
            if np.argmax(model_out)==0:
                label="1HandWave"
            if np.argmax(model_out)==1:
                label="2HandWave"
            if np.argmax(model_out)==2:
                label="Bending"
                
            cv2.putText(frame,label,(0,30),cv2.FONT_HERSHEY_DUPLEX,1,(0,0,255))
            cv2.imshow("frame",frame)
            c+=1
 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                 break
        else:
            break
       
  
# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows() 


